<a href="https://colab.research.google.com/github/ArshHp/LLM/blob/main/Embedding_Model_And_Retrieved_Chunks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:
#New Embedding Model for Portuguese Language

In [20]:
!pip install -q sentence-transformers

In [21]:
from sentence_transformers import SentenceTransformer

In [22]:
# Load the Instructor-XL model
embedding_model = SentenceTransformer("hkunlp/instructor-xl")


In [23]:
# Define instruction + Portuguese sentence
instruction = "Represent this sentence for semantic similarity"
sentence_pt = "O processo pode possuir mais de um localizador, mas nunca pode ficar sem um."

# Combine instruction and input sentence
input_text = [instruction, sentence_pt]

#input_text = f"{instruction}: {sentence_pt}"  # Merge into one string

# Get the embedding
embedding = embedding_model.encode(input_text)

In [24]:
# Print results
print(f"Embedding Shape: {embedding.shape}")  # Expected: (1, 768)
print(f"First 5 embedding values: {embedding[1][:5]}")  # Print first 5 dimensions

Embedding Shape: (2, 768)
First 5 embedding values: [ 0.02909243  0.00812454  0.01086273 -0.04523072 -0.06861272]


**Lets setup the ChromaDB to store the embeddings.**

In [25]:
!pip install -q chromadb

In [26]:
import chromadb
from langchain.text_splitter import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer

# Initialize the ChromaDB client
chroma_client = chromadb.PersistentClient(path="./chroma_db")  # Stores embeddings persistently

# Create or load a collection in ChromaDB
collection = chroma_client.get_or_create_collection(name="ptbr_documents")

# Define instruction for embedding generation
instruction = "Represent this sentence for semantic similarity"

In [27]:
# Sample Brazilian Portuguese document (replace with customer dataset here, manually crafted as sample)
document_text = """
No eproc, o processo pode possuir mais de um localizador, mas nunca pode ficar sem um.
É importante lembrar que, no caso de múltiplos localizadores, o principal ficará em negrito quando expandido o campo para inclusão ou exclusão.
Podemos pensar nos localizadores como etiquetas que são coladas e retiradas do processo durante toda sua tramitação.
"""

In [28]:
#Step 1: Split document into chunks #Doing basic splitter

text_splitter = RecursiveCharacterTextSplitter(chunk_size=200, chunk_overlap=50)
chunks = text_splitter.split_text(document_text)

# Step 2: Generate embeddings for each chunk
chunk_embeddings = [embedding_model.encode(f"{instruction}: {chunk}") for chunk in chunks]

# Step 3: Store embeddings in ChromaDB
for i, chunk in enumerate(chunks):
    collection.add(
        ids=[f"chunk_{i}"],  # Unique ID for each chunk
        documents=[chunk],  # Store the actual text
        embeddings=[chunk_embeddings[i].tolist()]  # Convert NumPy array to list
    )

print(f"✅ Successfully stored {len(chunks)} chunks in ChromaDB!")

✅ Successfully stored 3 chunks in ChromaDB!


In [29]:
query_text = "Onde está o localizador no processo?"
query_embedding = embedding_model.encode(f"{instruction}: {query_text}").tolist()

# Perform a similarity search in ChromaDB
results = collection.query(
    query_embeddings=[query_embedding],
    n_results=3  # Retrieve top 3 matches
)

# Print the most relevant chunks
print("🔍 Top matches:")
for i, doc in enumerate(results["documents"][0]):
    print(f"{i+1}. {doc}")


🔍 Top matches:
1. Podemos pensar nos localizadores como etiquetas que são coladas e retiradas do processo durante toda sua tramitação.
2. No eproc, o processo pode possuir mais de um localizador, mas nunca pode ficar sem um.
3. É importante lembrar que, no caso de múltiplos localizadores, o principal ficará em negrito quando expandido o campo para inclusão ou exclusão.


In [ ]:
#Use the above retrieved chunks to create the LLAMA 3.1 prompt and get the final response.